In [7]:
import pandas as pd
import numpy as np

import os
import sys
import warnings
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesse
 
# from dataset import engine, RFM, Churn, Engagement

In [8]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine


def create_db(user="root", password="Chenlu1974", server="localhost", database="transact"):
    SQLALCHEMY_DATABASE_URL = "mysql+pymysql://{}:{}@{}/{}".format(
        user, password, server, database
    )
    engine = create_engine(SQLALCHEMY_DATABASE_URL)

    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    Base = declarative_base()

    return engine, SessionLocal, Base


engine, SessionLocal, Base = create_db(password='msql1234')

with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT * FROM engagement;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()

In [3]:
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


data = df[df['engagement_date'] >= df['engagement_date'].max() - relativedelta(months=11)] \
	.groupby([df['engagement_date'].dt.to_period("M"), 'action_type']) \
	.agg(['count'])['customer_id'] \
	.reset_index() \
	.pivot(index="engagement_date", columns="action_type", values="count") \
	.reset_index() \
	.loc[:, ['engagement_date', 'converted', 'credentials', 'clicked', 'scrolled']] \
	.rename(columns={'engagement_date': 'date'}) \
	.astype({'date': str})
# df
data['convertedColor'] = ["hsl(229, 70%, 50%)" for i in range(len(data))]
data['credentialsColor'] = ["hsl(296, 70%, 50%)" for i in range(len(data))]
data['clickedColor'] = ["hsl(97, 70%, 50%)" for i in range(len(data))]
data['scrolledColor'] = ["hsl(229, 70%, 50%)" for i in range(len(data))]


result = data.to_dict(orient='records')
print(result)
# parsed = loads(result)
# print(dumps(parsed))

[{'date': '2024-01', 'converted': 15, 'credentials': 18, 'clicked': 48, 'scrolled': 102, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'}, {'date': '2024-02', 'converted': 219, 'credentials': 325, 'clicked': 1242, 'scrolled': 2465, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'}, {'date': '2024-03', 'converted': 158, 'credentials': 303, 'clicked': 1001, 'scrolled': 2020, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'}, {'date': '2024-04', 'converted': 128, 'credentials': 255, 'clicked': 920, 'scrolled': 1612, 'convertedColor': 'hsl(229, 70%, 50%)', 'credentialsColor': 'hsl(296, 70%, 50%)', 'clickedColor': 'hsl(97, 70%, 50%)', 'scrolledColor': 'hsl(229, 70%, 50%)'

In [4]:
# fig, ax = plt.subplots(figsize=(10, 8))
# df_new.plot(kind='line', stacked=True, linestyle='-', marker='o', ax=ax)
# plt.legend()

In [ ]:
with engine.connect() as db:
	query = sqlalchemy.text(
		'''
		SELECT *
		FROM engagement e
		ORDER BY e.engagement_date DESC
		LIMIT 10;
		''')
	df = pd.DataFrame(db.execute(query).fetchall())
	db.close()

df['engagement_date'] = df['engagement_date'].astype(str)
df = df.loc[:, ['customer_id', 'engagement_date', 'action_type', 'feedback_score']]
df = df.rename(columns={'customer_id': 'id', 'engagement_date': 'date', 'action_type': 'action', 'feedback_score': 'score'})
# df.to_dict(orient='records')
df

,engagement_id,campaign_id,customer_id,engagement_date,action_type,device_type,feedback_score,conversion_value
0,10205,0276,1904,2024-12-28,scrolled,laptop,2,0.0
1,11745,0219,1071,2024-12-28,credentials,laptop,1,0.0
2,12282,0113,1660,2024-12-28,scrolled,laptop,2,0.0
3,1337,0010,0134,2024-12-28,scrolled,mobile,1,0.0
4,1025,0154,1805,2024-12-28,scrolled,desktop,1,0.0
5,11966,0239,0130,2024-12-28,clicked,mobile,3,0.0
6,10272,0075,1931,2024-12-28,scrolled,laptop,2,0.0
7,11465,0006,1048,2024-12-28,credentials,desktop,2,0.0
8,13682,0009,1679,2024-12-28,scrolled,laptop,2,0.0
9,11908,0114,1435,2024-12-28,clicked,laptop,3,0.0
